In [ ]:
## A. Moore
## this notebook was created as an easy and interactive way to upload an image from the user's computer, generate image tags, and utilize deepseek R1 to generate a haiku about said image

In [ ]:
from PIL import Image
import os
import re
from transformers import BlipProcessor, BlipForConditionalGeneration
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import UserMessage
from azure.core.credentials import AzureKeyCredential

In [ ]:
## Importing the tagging model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

## github models is powered by azure
endpoint = "https://models.inference.ai.azure.com"
model_name = "DeepSeek-R1"

## add your github token to the environment
token = os.environ["GITHUB_TOKEN"]


In [ ]:
## Generate the image tags by specifying a file path

image = Image.open("<file path to image>").convert("RGB")
inputs = processor(image, return_tensors="pt")
out = model.generate(**inputs)
caption = processor.decode(out[0], skip_special_tokens=True)
print(caption)


In [ ]:
## Pass the image caption to deespseek to generate the haiku

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        UserMessage(content=f"write a haiku about {caption}"),
    ],
    max_tokens=1000,
    model=model_name
)



content = response.choices[0].message.content
print(content)

#Used to remove the text between the think tags i.e. the model "reasoning"
cleaned_text = re.sub(r"<think>.*?</think>", "", content, flags=re.DOTALL)

#This expression removes the sentence contained after the haiku
cleaned_text = re.sub(r"\*", "", cleaned_text)

# Extract haiku using regex
print(cleaned_text)


In [ ]:
cleaned_text = re.sub(r"<think>.*?</think>", "", content, flags=re.DOTALL)
cleaned_text = re.sub(r"\*[^*]*$", "", cleaned_text, flags=re.DOTALL)
cleaned_text = re.sub(r"\*.*?", "", cleaned_text)
print(cleaned_text)